# A. Cleaning data:

## A.1 Wide to Long

### A.1.1 Import

In [20]:
import pandas as pd
import os
data_folder = '../../data/examples/module_3'
os.listdir(data_folder)

['gini.csv', 'pisa.csv']

In [21]:
df_gini = pd.read_csv(f"{data_folder}/gini.csv")
df_gini_clean = df_gini.drop(['Unnamed: 0'], axis = 'columns')
df_gini_clean = df_gini_clean.rename(columns = {"Country Name": 'country'})
df_gini_clean.head(2)

,country,2000,2001,2002,2003,2004,2005,2006,2007,2008,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,Australia,..,33.5,..,33.5,33.1,..,..,..,35.4,...,..,34.4,..,33.7,..,34.3,..,..,..,..
1,Austria,29,..,..,29.5,29.8,28.7,29.6,30.6,30.4,...,30.8,30.5,30.5,30.8,29.7,30.8,30.2,29.8,30.7,..


In [22]:
df_pisa = pd.read_csv(f"{data_folder}/pisa.csv")
df_pisa_clean = df_pisa.copy().drop(['Unnamed: 0'], axis = 'columns')
df_pisa_clean.head(2)

,country,2000,2003,2006,2009,2012,2015,2018,2022,region,subregion
0,Australia,528.0,525.0,513.0,515.0,512.0,503.0,503.0,498.0,Oceania,Australia and New Zealand
1,Austria,492.0,491.0,490.0,470.0,490.0,485.0,484.0,480.0,Europe,Western Europe


### A.1.2 Drop or set index all non-year columns

In [23]:
df_pisa_clean = df_pisa_clean.set_index(['country', 'subregion', 'region'])
df_pisa_clean.head(2)

,,,2000,2003,2006,2009,2012,2015,2018,2022
country,subregion,region,,,,,,,,
Australia,Australia and New Zealand,Oceania,528.0,525.0,513.0,515.0,512.0,503.0,503.0,498.0
Austria,Western Europe,Europe,492.0,491.0,490.0,470.0,490.0,485.0,484.0,480.0


In [24]:
df_gini_clean = df_gini_clean.set_index('country')
df_gini_clean.head(2)

,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
country,,,,,,,,,,,,,,,,,,,,,
Australia,..,33.5,..,33.5,33.1,..,..,..,35.4,..,...,..,34.4,..,33.7,..,34.3,..,..,..,..
Austria,29,..,..,29.5,29.8,28.7,29.6,30.6,30.4,31.5,...,30.8,30.5,30.5,30.8,29.7,30.8,30.2,29.8,30.7,..


### A.1.3 Wide to long: stack

In [29]:
df_gini_long = df_gini_clean.stack().reset_index().rename(columns = {"level_1": 'date', 0:'gini'})
df_gini_long.head(2)

,country,date,gini
0,Australia,2000,..
1,Australia,2001,33.5


In [36]:
df_pisa_long = df_pisa_clean.stack().reset_index().rename(columns = {"level_3": 'date', 0:'pisa'})
df_pisa_long.head(2)

,country,subregion,region,date,pisa
0,Australia,Australia and New Zealand,Oceania,2000,528.0
1,Australia,Australia and New Zealand,Oceania,2003,525.0


### A.1.M Merge

In [37]:
df_merge = pd.merge(df_pisa_long, df_gini_long, on = ['country','date'])
df_merge.head(2)

,country,subregion,region,date,pisa,gini
0,Australia,Australia and New Zealand,Oceania,2000,528.0,..
1,Australia,Australia and New Zealand,Oceania,2003,525.0,33.5


In [38]:
df_merge = pd.merge(
    df_pisa_long.set_index(['country','date']), 
    df_gini_long.set_index(['country','date']), 
    right_index = True,
    left_index = True
)
df_merge.head(2)

subregion   region   pisa  gini
country   date                                                 
Australia 2000  Australia and New Zealand  Oceania  528.0    ..
          2003  Australia and New Zealand  Oceania  525.0  33.5

## A.2 Fill na

### A.2.1 Wide to long: Melt

In [106]:
df_health = pd.read_csv(f"{data_folder}/health_expenditure_raw.csv", skiprows=4)
df_health = df_health.drop(['Country Name', 'Indicator Code'], axis='columns')
df_health.head(2)

,Country Code,Indicator Name,1960,1961,1962,1963,1964,1965,1966,1967,...,2016,2017,2018,2019,2020,2021,2022,2023,2024,Unnamed: 69
0,ABW,Current health expenditure (% of GDP),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AFE,Current health expenditure (% of GDP),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,6.06777,5.928989,5.789177,5.812624,6.035792,6.043925,5.728123,NaN,NaN,NaN


In [109]:
df_health_long = df_health.melt(id_vars = ['Country Code', 'Indicator Name'], value_name = 'healthexp_gdp', var_name = 'date')
df_health_long.head(2)

,Country Code,Indicator Name,date,healthexp_gdp
0,ABW,Current health expenditure (% of GDP),1960,NaN
1,AFE,Current health expenditure (% of GDP),1960,NaN


In [113]:
df_health_long = df_health_long.query("date !='Unnamed: 69'")
df_health_long.head(2)

,Country Code,Indicator Name,date,healthexp_gdp
0,ABW,Current health expenditure (% of GDP),1960,NaN
1,AFE,Current health expenditure (% of GDP),1960,NaN


In [114]:
df_health_long['date'] = df_health_long['date'].astype(int)
df_health_long.head(2)

,Country Code,Indicator Name,date,healthexp_gdp
0,ABW,Current health expenditure (% of GDP),1960,NaN
1,AFE,Current health expenditure (% of GDP),1960,NaN


In [115]:
df_health_long = df_health_long.set_index(['Country Code', 'date']).drop(['Indicator Name'], axis='columns')
df_health_long.head(2)

,,healthexp_gdp
Country Code,date,
ABW,1960,NaN
AFE,1960,NaN


In [116]:
df_health_long = df_health_long.sort_index()
df_health_long.head(2)

healthexp_gdp
Country Code date               
ABW          1960            NaN
             1961            NaN

In [117]:
df_health_long['healthexp_gdp'].isna().sum()

np.int64(11827)

### A.2.2 forward fill

In [144]:
df_heal = df_health_long.reset_index().rename({"Country Code" : 'country'}, axis='columns')
df_heal.head(2)

,country,date,healthexp_gdp
0,ABW,1960,NaN
1,ABW,1961,NaN


### One country: easy

In [145]:
df_zmb = df_heal.query("country == 'ZMB'").tail(4)
df_zmb

,country,date,healthexp_gdp
17221,ZMB,2021,6.639514
17222,ZMB,2022,5.255923
17223,ZMB,2023,NaN
17224,ZMB,2024,NaN


In [146]:
df_zmb['healthexp_gdp'] = df_zmb['healthexp_gdp'].ffill()
df_zmb

,country,date,healthexp_gdp
17221,ZMB,2021,6.639514
17222,ZMB,2022,5.255923
17223,ZMB,2023,5.255923
17224,ZMB,2024,5.255923


### Multiple countries: group by

In [140]:
df_heal['healthexp_gdp'].isna().sum()

np.int64(11827)

In [143]:
df_heal['healthexp_gdp'] = df_heal.groupby(['country'])['healthexp_gdp'].ffill()
df_heal['healthexp_gdp'].isna().sum()

np.int64(0)